In [ ]:
#| output: false
using Cropbox, Printf, Unitful

In [ ]:
@system VaporPressure begin
    # Campbell and Norman (1998), p 41 Saturation vapor pressure in kPa
    a => 0.611 ~ preserve(u"kPa", parameter)
    b => 17.502 ~ preserve(parameter)
    c => 240.97 ~ preserve(parameter) # °C

    es(a, b, c; T(u"°C")): saturation => (t = Cropbox.deunitfy(T); a*exp((b*t)/(c+t))) ~ call(u"kPa")
    ea(es; T(u"°C"), RH(u"percent")): ambient => es(T) * RH ~ call(u"kPa")
    D(es; T(u"°C"), RH(u"percent")): deficit => es(T) * (1 - RH) ~ call(u"kPa")
    RH(es; T(u"°C"), VPD(u"kPa")): relative_humidity => 1 - VPD / es(T) ~ call(u"NoUnits")

    # slope of the sat vapor pressure curve: first order derivative of Es with respect to T
    Δ(es, b, c; T(u"°C")): saturation_slope_delta => (e = es(T); t = Cropbox.deunitfy(T); e*(b*c)/(c+t)^2 / u"K") ~ call(u"kPa/K")
    s(Δ; T(u"°C"), P(u"kPa")): saturation_slope => Δ(T) / P ~ call(u"K^-1")
end

@system Weather begin
    vp(context): vapor_pressure ~ ::VaporPressure

    PFD:           photon_flux_density      => 1500 ~ preserve(parameter, u"μmol/m^2/s")
    CO2:           carbon_dioxide           => 420  ~ preserve(parameter, u"μmol/mol")
    RH:            relative_humidity        => 60   ~ preserve(parameter, u"percent")
    T_air:         air_temperature          => 25   ~ preserve(parameter, u"°C")
    Tk_air(T_air): absolute_air_temperature         ~ track(u"K")
    wind:          wind_speed               => 2.0  ~ preserve(parameter, u"m/s")
    P_air:         air_pressure             => 101.3 ~ preserve(parameter, u"kPa")

    e_sat(T_air, vp.es):    saturation_vapor_pressure             => es(T_air)       ~ track(u"kPa")
    e_air(T_air, RH,  ea = vp.ea):    ambient_vapor_pressure                => ea(T_air, RH)   ~ track(u"kPa")
    VPD(T_air, RH, D=vp.D):      vapor_pressure_deficit                => D(T_air, RH)    ~ track(u"kPa")
    VPD_Δ(T_air, Δ=vp.Δ):        vapor_pressure_saturation_slope_delta => Δ(T_air)        ~ track(u"kPa/K")
    VPD_s(T_air, P_air, s=vp.s): vapor_pressure_saturation_slope       => s(T_air, P_air) ~ track(u"K^-1")
end

@system Environment(Weather, Controller) 

Check the default parameter values at the standard conditions of the systems: `Weather` and `Environment`

In [ ]:
parameters(Weather);

We set up variable ranges and groups for simulating environmental conditions.

In [ ]:
co2_xstep = :Weather => :CO2   => 10:10:1500;
pfd_xstep = :Weather => :PFD   => 0:20:2000;
ta_xstep  = :Weather => :T_air => 0:1:50;
rh_xstep  = :Weather => :RH    => 0:1:100;

co2_group = :Weather => :CO2   => [1000, 400, 250];
pfd_group = :Weather => :PFD   => 1800:-400:600;
ta_group  = :Weather => :T_air => 40:-5:10;
rh_group  = :Weather => :RH    => 80:-30:20;

Plot and examine the temperature responses of saturation vapor pressure ($e_s$), ambient vapor pressure ($e_a$), and *VPD* at different *RH* conditions.

In [ ]:
visualize(Environment, :T_air, [:e_sat, :e_air]; xstep = ta_xstep, ylim = (0, 12), kind = :line)

Generate VPD response over time at different relative humidity using `rh_group`.

In [ ]:
visualize(Environment, :T_air, :VPD; xstep = ta_xstep, group = rh_group, ylim = (0, 12), kind = :line)

:::

::: {#tip-cropbox-call .callout-tip}
#### `call`, `solve`, and `bisect` variables in `Cropbox`

Here we are introduced some new method variables in `Cropbox`.

#### Call {.unnumbered}

`call` makes a *callable* variable which works like a function. Variables declared in the argument list before `;` are mapped to the existing variables available in the system. Variables declared in the argument list after `;` need to be supplied explicitly each time the `call` variable (i.e., function) is called outside the system definition.

In the example below, `f` is a `call` variable that is dependent on `a` in the system and callable with `x` outside the system. This is somewhat similar to using *keyword* argument after `;` to change a parameter of the system by assigning a value to a keyword. Look at how it can called in the declaration of variable `f2` inside the system as well as outside the system definition.

In [ ]:
@system SCall(Controller) begin
    a       => 1 ~ preserve
    f(a; x) => a + x ~ call
    f2(f)   => f(2) ~ track
end
simulate(SCall)
s = instance(SCall)
s.f(2)
@look s.f(3; a = 5);

#### Solve {.unnumbered}

As we discussed in @sec-photosynthesis for solving a quadratic equation, `solve` derives a symbolic solution of polynomial equation via [SymPy.jl](https://github.com/JuliaPy/SymPy.jl). The equation is solved for the name of variable itself. For example, the variable `x` below is resolved as -1 as $x = -1$ for the equation $x^2 + 2x + 1 = (x + 1)^2$. Trying solving a cubic equation as well.

In [ ]:
@system SSolve(Controller) begin
    a => 1 ~ preserve
    b => 2 ~ preserve
    c => 1 ~ preserve
    x(a, b, c) => a*x^2 + b*x + c ~ solve
end
simulate(SSolve);

#### Bisect {.unnumbered}

`bisect` also derives a solution of equations. While `solve` can find an analytical solution only when the equation is solvable via symbolic algebra, `bisect` relies on numerical optimizations which are more generally applicable to more types of equations. The downside is that it is much slower and can still fail especially when the given boundary of solution is not correct. The boundary of solution should be given by `lower` and `upper` tags.

In [ ]:
@system SBisect(Controller) begin
    x(x) => x - 1 ~ bisect(lower = 0, upper = 2)
end
simulate(SBisect);

In [ ]:
#| output: false
using Unitful
    # parameters
    λ = 2.454u"MJ/kg"
    c_p = 1.013u"kJ/kg/K"
    ρ_a = 1.204u"kg/m^3"
    s = 0.145u"kPa/K"
    γ = 66.1u"Pa/K"
    #Rn = 400u"W/m^2" # (R_n - G)
    Rn = 10u"MJ/m^2/d"
    G = 0u"MJ/m^2/d"   # Soil heat flux
    D = 1.0u"kPa"
    R = 8.314u"J/mol/K"
    T_air = 20.0u"°C" |> u"K"
    P = 101.3u"kPa"
    H2O_m = 18.01528u"g/mol"

# Function to calculate ET using Penman-Monteith equation
function penmon_ET(g_H, g_L)
    # Canopy conductance for water vapor, g_v, is calculated as the parallel combination of g_L and g_H
    # It is easier to calculate the resistances first and then convert to conductance
    # r_v = ((r_L) + (r_H))
    g_W = (g_L^-1 + g_H^-1)^-1
    # Penman-Monteith equation
    ET = (s * (Rn - G) + ρ_a * c_p * g_H * D) / (λ * (s + γ * (g_H / g_W))) |> u"g/m^2/s"
    g_Wm = g_W*P/(R*T_air)
    ET_m = ET/H2O_m |> u"mmol/m^2/s"
    ET_daily = ET * (60*60*24)u"s/d" |> u"kg/m^2/d"
    return ET, g_Wm, ET_daily
end
# Scenario a: Forest
g_H_forest = 0.2u"m/s"
g_L_forest = 0.03u"m/s"
ET_forest, gv_forest, ET_forest_daily = penmon_ET(g_H_forest, g_L_forest)
println("ET for Forest: ", ET_forest)
println("g_W for Forest: ", gv_forest |> u"mol/m^2/s")
println("ET_daily for Forest: ", ET_forest_daily)
println()

# Scenario b: Grassland
g_H_grassland = 0.01u"m/s"
g_L_grassland = 0.03u"m/s"
ET_grassland, gv_grassland, ET_grassland_daily = penmon_ET(g_H_grassland, g_L_grassland)
println("ET for Grassland: ", ET_grassland)
println("g_W for Grassland: ", gv_grassland |> u"mol/m^2/s")
println("ET_daily for Grassland: ", ET_grassland_daily)
println()

# Scenario c: When surface is wet, canopy conductance approaches infinity and g_v ≈ g_H
g_L_forest = (Inf)*u"m/s"
ET_forest_wet, gv_forest_wet = penmon_ET(g_H_forest, g_L_forest)
println("ET for Wet Forest: ", ET_forest_wet)
println("g_W for Wet Forest: ", gv_forest_wet |> u"mol/m^2/s")
println()

# Scenario d: Wet surface for Grassland (g_v approaches infinity)
g_L_grassland = (Inf)*u"m/s"
ET_grassland_wet, gv_grassland_wet = penmon_ET(g_H_grassland, g_L_grassland)
println("ET for Wet Grassland: ", ET_grassland_wet)
println("g_W for Wet Grassland: ", gv_grassland_wet |> u"mol/m^2/s")
println()

In [ ]:
"""
Calculate daily evapotranspiration from a plant canopy using the Penman-Monteith equation

Returns:
- NamedTuple containing ET results
"""
function calculate_canopy_evapotranspiration(;
    canopy_length = 1.0,         # meters
    air_temp = 25.0,               # °C
    relative_humidity = 60.0,      # percent
    wind_speed = 2.0,              # m/s
    solar_radiation = 20.0,        # MJ/m²/d
    atmospheric_pressure = 101.3,  # kPa
    canopy_conductance = 600.0,    # mmol/m²/s (stomatal conductance)
    canopy_height = 2.0            # meters
)
    
    # Canopy surface area (m²)
    surface_area = canopy_length^2
    
    # Psychrometric constant (kPa/K)
    γ = 0.665 * atmospheric_pressure
    
    # Saturation vapor pressure (kPa) - Tetens equation
    e_sat = 0.6108 * exp(17.27 * air_temp / (air_temp + 237.3))
    
    # Actual vapor pressure (kPa)
    RH_fraction = relative_humidity / 100
    e_actual = RH_fraction * e_sat
    
    # Vapor pressure deficit (kPa)
    vpd = e_sat - e_actual
    
    # Slope of saturation vapor pressure curve (kPa/K)
    Δ = 4098 * e_sat / (air_temp + 237.3)^2
    s = Δ / atmospheric_pressure  # saturation slope (K^-1)
    
    # Net radiation (MJ/m²/d) - for vegetation, assume 80% of incoming radiation
    R_n = 0.8 * solar_radiation
    
    # Aerodynamic conductance calculations (Campbell & Norman, 1998)
    # ga = 1/ra = (k²*u) / [ln((z-d)/z0) * ln((z-d)/z0h)]
    d = 0.67 * canopy_height  # zero plane displacement
    z0 = 0.1 * canopy_height  # roughness length for momentum
    z0h = 0.1 * z0            # roughness length for heat
    z = canopy_height + 1.0                   # measurement height (m)
    k = 0.41                  # von Karman constant
    
    if wind_speed > 0
        ga = (k^2 * wind_speed) / (log((z - d) / z0) * log((z - d) / z0h))
    else
        ga = air_temp / 208  # default value for calm conditions
    end
    
    # Convert conductance to resistance
    ra = 1 / ga  # aerodynamic resistance (s/m)
    
    # Convert canopy conductance from mmol/m²/s to m/s (Jones, 2013)
    # Conversion: gs(m/s) = gs(mmol/m²/s) * R * T / P * 1e-3
    R_gas = 8.314  # J/mol/K (universal gas constant)
    T_kelvin = air_temp + 273.15  # temperature in K
    P_pascal = atmospheric_pressure * 1000  # pressure in Pa
    
    # Convert from mmol/m²/s to mol/m²/s, then to m/s
    canopy_conductance_ms = canopy_conductance * 1e-3 * R_gas * T_kelvin / P_pascal
    rc = 1 / canopy_conductance_ms  # canopy resistance (s/m)
    
    # Latent heat of vaporization (MJ/kg)
    λ = 2.45
    
    # Air density (kg/m³) - approximation at standard conditions
    ρ = 1.225 * (293 / (air_temp + 273.15)) * (atmospheric_pressure / 101.3)
    
    # Specific heat of air (MJ/kg/K)
    cp = 0.001013
    
    # Penman-Monteith equation components
    # Radiation term
    ET_rad = s / (s + γ * (1 + rc / ra)) * (R_n / λ) * 86400  # Convert s to day
    # Aerodynamic term
    ET_aero = (γ / (s + γ * (1 + rc / ra))) * (ρ * cp * vpd / ra) / λ * 86400  # Convert s to day
    
    # Total evapotranspiration (mm/day)
    ET_daily = (ET_rad + ET_aero) 
    
    # Total water loss from canopy (liters/day)
    water_loss_volume = (ET_daily / 1000) * surface_area * 1000  # L/d
    
    return (
        evapotranspiration_mm = ET_daily,
        surface_area = surface_area,
        water_loss_volume = water_loss_volume,
        radiation_component = ET_rad,
        aerodynamic_component = ET_aero,
        vapor_pressure_deficit = vpd,
        aerodynamic_conductance = ga,
        canopy_conductance_mmol = canopy_conductance,
        canopy_conductance_ms = canopy_conductance_ms,
        aerodynamic_resistance = ra,
        canopy_resistance = rc,
    )
end

In [ ]:
using Printf
# Example calculation for canopy
println("\n=== Plant Canopy Evapotranspiration Calculator ===")

canopy_result = calculate_canopy_evapotranspiration()

@printf "Surface area: %.2f m²\n" canopy_result.surface_area
@printf "Daily evapotranspiration: %.2f mm/d\n" canopy_result.evapotranspiration_mm
@printf "Daily water loss: %.2f L/d\n" canopy_result.water_loss_volume
@printf "Radiation component: %.4f mm/d\n" canopy_result.radiation_component
@printf "Aerodynamic component: %.4f mm/d\n" canopy_result.aerodynamic_component
@printf "Vapor pressure deficit: %.4f kPa\n" canopy_result.vapor_pressure_deficit
@printf "Aerodynamic conductance: %.4f m/s\n" canopy_result.aerodynamic_conductance
@printf "Canopy conductance: %.1f mmol/m²/s\n" canopy_result.canopy_conductance_mmol
@printf "Canopy conductance: %.4f m/s\n" canopy_result.canopy_conductance_ms
@printf "Aerodynamic resistance: %.1f s/m\n" canopy_result.aerodynamic_resistance
@printf "Canopy resistance: %.1f s/m\n" canopy_result.canopy_resistance